ЗАГРУЗКА ДАННЫХ

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

#Загрузка датасетов
submission_data_test = pd.read_csv(r"C:\Users\User\Desktop\test project\submission_data_test.csv")
submission_data_train = pd.read_csv(r"C:\Users\User\Desktop\test project\submissions_data_train.csv")
events_data_test = pd.read_csv(r"C:\Users\User\Desktop\test project\events_data_test.csv")
events_data_train = pd.read_csv(r"C:\Users\User\Desktop\test project\event_data_train.csv")



In [9]:
def filtered_data(data, days=2):
    '''Фильтрация данных с порогом 2 дня от начала обучения'''
    data['timestamp'] = data['timestamp'].astype('datetime64[s]').dt.date

    #Создаем колонку с первым действием пользователя
    r_side_to_merge = data.groupby('user_id', as_index=False) \
                        .agg({'timestamp':'min'}) \
                        .rename({'timestamp':'min_timestamp'}, axis=1)
    
    #Объединяем и фильтруем по времени + 2 дня от даты старта
    data = data.merge(r_side_to_merge, on='user_id', how='outer')
    data = data[data.timestamp < data.min_timestamp + pd.DateOffset(days, 'D')]
    
    #Проверяем что все пользователи на месте
    assert data.user_id.nunique() == data.user_id.nunique()
    return data.drop('min_timestamp', axis=1)


In [10]:
def create_base_feature(submission_data, events_data):
    '''Выделение в числовые признаки изначальные данные и объединение в одну таблицу'''
    submission_data = submission_data.pivot_table(columns='submission_status', 
                                 values='step_id', 
                                 aggfunc='count', 
                                 index='user_id', 
                                 fill_value=0) \
                                .reset_index() \
                                .rename_axis('', axis=1)
    events_data = events_data.pivot_table(columns='action', 
                             values='step_id', 
                             aggfunc='count', 
                             index='user_id',
                             fill_value=0) \
                             .reset_index() \
                             .rename_axis('', axis=1)
    
    #Объединяем
    merged_df = submission_data.merge(events_data, 
                                           on='user_id', 
                                           how='outer') \
                                           .fillna(0)
    
    #Проверяем что все пользователи на остались в данных
    assert events_data.user_id.nunique() == merged_df.user_id.nunique()
    return merged_df

In [18]:
def create_target(submission_data, threshold=40):
    '''Создаем таргет для расчета что будет считаться успешным прохождением теста. По умолчанию принимаем 40 правильных ответов как условие прохождения
    Значениsubmission_data_test['correct_num'].hist(bins=50)
    '''

    #Создаем колонку с количеством правильных ответов
    submission_data = submission_data[submission_data['submission_status'] == 'correct'].groupby('user_id') \
                                            .agg({'submission_status':'count'}) \
                                            .reset_index()  \
                                            .rename({'submission_status':'correct_num'}, axis=1)
    
    #Фильтруем по ранее созданной колонке
    submission_data['passed_course'] = (submission_data['correct_num'] > threshold).astype('int')
    return submission_data.drop('correct_num', axis=1)


In [12]:
def time_features(event_data):
    '''Создаем колонку часов посвященных на обучению'''
    event_data['day'] = event_data['timestamp'].astype('datetime64[s]').dt.date
    event_data['date'] = event_data['timestamp'].astype('datetime64[s]')

    event_data = event_data.groupby('user_id') \
                                    .agg({'date':['max', 'min'], 'day':'nunique'}) \
                                    .rename(columns={'nunique':'days'}) \
                                    .droplevel(0, axis=1)
    event_data['hours'] = (event_data['max'] - event_data['min']).astype('timedelta64[h]')

    return event_data.drop(['max', 'min'], axis=1).reset_index()


In [13]:
def tries_feature(submission_data):
    submission_data = submission_data.groupby('user_id', as_index=False) \
                                    .agg({'step_id':'nunique'}) \
                                    .rename({'step_id':'steps_tried'}, axis=1)
    return submission_data

In [14]:
def correct_ratio(submission_data):
    '''Добавляем колонку соотношения верных и неверных ответов пользователя'''
    #Группируем и создаем признак
    submission_data = submission_data.pivot_table(index='user_id', columns='submission_status', values='step_id', aggfunc='count', fill_value=0).reset_index()
    submission_data['correct_ratio'] = submission_data['correct'] / (submission_data['correct'] + submission_data['wrong'])
    return submission_data.drop(['correct', 'wrong'], axis=1).rename_axis('', axis=1)

In [35]:
def create_df(submission_data, events_data):
    '''Функция для формирования Х и у датасетов'''
    # Фильтруем данные по первым 2 дням обучения для каждого студента
    submission_data = filtered_data(submission_data)
    events_data = filtered_data(events_data)

    # Добавляем базовые признакми и объединяем в единую таблицу
    merged_df = create_base_feature(submission_data, events_data=events_data)

    #Создаем целевую переменную Y
    temp_y = create_target(submission_data, threshold=40)

    #Создаем доп признаки по времени и соотношению правильных/неправильных ответов
    time_column = time_features(events_data)
    ratio_features = correct_ratio(submission_data)

    #Объединяем с основной таблицей дополнительные вычисленные признаки
    merged_df = merged_df.merge(time_column, on='user_id', how='outer').fillna(0)
    merged_df = merged_df.merge(ratio_features, on='user_id', how='outer').fillna(0)
    merged_df = merged_df.merge(temp_y, on='user_id', how='outer').fillna(0)

    #Создаем целевую переменную X
    y = merged_df['passed_course'].map(int)
    X = merged_df.drop(columns='passed_course')
    
    return X, y

In [16]:
def create_test_df(submission_data, events_data):
    '''Функция для формирования Х и у датасетов'''
    # Фильтруем данные по первым 2 дням обучения для каждого студента
    submission_data = filtered_data(submission_data)
    events_data = filtered_data(events_data)

    # Добавляем базовые признакми и объединяем в единую таблицу
    merged_df = create_base_feature(submission_data, events_data)

    #Создаем целевую переменную Y
    y = create_target(submission_data, threshold=40)

    #Создаем доп признаки по времени и соотношению правильных/неправильных ответов
    time_column = time_features(events_data)
    ratio_features = correct_ratio(submission_data)

    #Объединяем с основной таблицей дополнительные вычисленные признаки
    merged_df = merged_df.merge(time_column, on='user_id', how='outer').fillna(0)
    X = merged_df.merge(ratio_features, on='user_id', how='outer').fillna(0)


    return X

In [36]:
#Создание треннировочного и тестового датасетов
X_train, y = create_df(submission_data_train, events_data_train)
#X_test = create_test_df(submission_data_test, events_data_test)

In [38]:
def random_with_grid(train_data, y, size=0.20):
    
    """Поиск наилучших параметров для RandomForest, обучаясь на тренировочной выборке.
    Можно изменять или добавлять различные параметры. Может долго вычисляться."""
    
    X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=size, random_state=42)
    
    param_grid = {'randomforestclassifier__n_estimators': range(20, 51, 3), 
                  'randomforestclassifier__max_depth': range(5, 14)}
    
    pipe = make_pipeline(RandomForestClassifier())
    pipe.fit(X_train, y_train)
    grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)
    grid.fit(X_train, y_train)
    print(f"Наилучшие параметры: {grid.best_params_}")
    
    ypred_prob = grid.predict_proba(X_test)
    
    roc_score = roc_auc_score(y_test, ypred_prob[:, 1])
    score = grid.score(X_test, y_test)
    print(f"Правильность на тестовом наборе: {score:.2f}")
    print(roc_score)

In [39]:
model = random_with_grid(X_train, y)
model

Наилучшие параметры: {'randomforestclassifier__max_depth': 5, 'randomforestclassifier__n_estimators': 20}
Правильность на тестовом наборе: 1.00
1.0
